In [314]:
import requests
import os
import urllib
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm

In [239]:
api_url = 'https://sandbox86.tactiorpm7000.com/tactio-clinician-api/1.1.4/'
token_url = 'https://sandbox86.tactiorpm7000.com/token.php'
token_request = {'grant_type' : 'password',
               'client_id' : '083e9a44a763473fbeb62fbf90b74551',
               'client_secret' : 'ba09798f0921456e8b4e5e4588ea536d',
               'username' : 'tactioClinician',
               'password' : 'tactio'}
token = requests.post(token_url, data=token_request)
token = token.json()
token_hdr = {'Authorization' : f'Bearer {token["access_token"]}'}

In [247]:
patient_num = requests.get(f'{api_url}Patient', headers=token_hdr).json()
parsed_url = urllib.parse.urlparse(patient_num['link'][2]['url'])
max_page_num = int(urllib.parse.parse_qs(parsed_url.query)['page'][0])
pids = []
for i in range(1,max_page_num+1):
    r = requests.get(f'{api_url}Patient?page={i}', headers=token_hdr)
    pids += [entry['resource']['id'] for entry in r.json()['entry']]

In [315]:
# for pid in pids:
#     obs_params = {'subject' : 'bced140a-0513-4873-af91-874fbec94e89'}
#     pdata = requests.get(f'{api_url}Observation', 
#                          params=obs_params, 
#                          headers=token_hdr)
#     pdata = pdata.json()
#     print(pdata)
#     break

glvls = []
for pid in tqdm(pids, total=len(pids)):    
    obs_params = {'subject' : pid}
    pdata = requests.get(f'{api_url}Observation', 
                         params=obs_params, 
                         headers=token_hdr).json()
    for entry in pdata['entry']:
        if entry['resource']['code']['text'] == 'glucose':
            glvl = entry['resource']['valueQuantity']['value']
            glvls.append(glvl)
glvls = np.array(glvls, dtype=np.float64)

plt.scatter(glvls, [0]*glvls.size)
plt.show()

KeyError: 'entry'